# SK RAG Pattern Foundational Concepts - Redis Cache

Learning objectives:

- RAG pattern foundational concepts
- Redis cache as a vector database

## Redis setup

### Running Redis in a local container:

- docker pull `redis/redis-stack:latest`
  - Note this version of redis includes the `RedisSearch` module
- Then execute: 
  - `docker run -d --name redis-stack -p 6379:6379 -p 8001:8001 redis/redis-stack:latest`

Connection string:
- `PG_CONN_STR="Host=<server>;Port=5432;Database=<database>;Username=<user>;Password=<password>"`

### Running from Azure Redis Cache

Connection string:

- `REDIS_CONN_STR=localhost`

## Setup

### Load required .NET packages and supporting constants, classes, etc.

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Core, 1.4.0"
#r "nuget: Microsoft.SemanticKernel.Plugins.Memory, 1.4.0-alpha"
#r "nuget: Microsoft.SemanticKernel.Connectors.Redis, 1.4.0-alpha"
#r "nuget: StackExchange.Redis"
#r "nuget: dotenv.net"

using System;

using System.IO;
using System.Text;
using System.Text.RegularExpressions;
using System.Text.Json;
using System.Text.Json.Serialization;
using StackExchange.Redis;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.SemanticKernel.Connectors.Redis;
using Microsoft.SemanticKernel.Memory;
using Microsoft.SemanticKernel.Plugins.Memory;

using dotenv.net;
using InteractiveKernel = Microsoft.DotNet.Interactive.Kernel;

#!import Models/Models.cs

const int ADA_EMBEDDING_SIZE = 1536;
const string MemoryCollectionName = "RedisCollection";

### Read the API Key and endpoints from environment variables or the .env file

In [ ]:
// Load the .env file
DotEnv.Load();

// Get the OpenAI deployment name, endpoint, and key from the environment variables
var deploymentName = Environment.GetEnvironmentVariable("GPT_OPENAI_DEPLOYMENT_NAME");
var endpoint = Environment.GetEnvironmentVariable("GPT_OPENAI_ENDPOINT");
var apiKey = Environment.GetEnvironmentVariable("GPT_OPENAI_KEY");
var redis_conn_str = Environment.GetEnvironmentVariable("REDIS_CONN_STR");
var adaDeploymentName = "ada";

### Get a kernel instance configured for text completions and embeddings

In [ ]:
// I'm using a RAM stored Vector DB, but I can switch providers like Azure Search, DuckDB, SQLite, etc.
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

var kernel = Kernel.CreateBuilder()
    .AddAzureOpenAIChatCompletion(deploymentName, endpoint, apiKey)
    .AddAzureOpenAITextEmbeddingGeneration(adaDeploymentName, endpoint, apiKey)
    .Build();

In [ ]:
#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102

ConnectionMultiplexer connectionMultiplexer = await ConnectionMultiplexer.ConnectAsync(redis_conn_str);
IDatabase database = connectionMultiplexer.GetDatabase();
IMemoryStore memoryStore = new RedisMemoryStore(database, vectorSize: ADA_EMBEDDING_SIZE);

var embeddingGenerator = new AzureOpenAITextEmbeddingGenerationService(adaDeploymentName, endpoint, apiKey);

// The combination of the text embedding generator and the memory store makes up the 'SemanticTextMemory' object used to
// store and retrieve memories.
SemanticTextMemory textMemory = new(memoryStore, embeddingGenerator);

## Ingestion

### Read the files and chunk them by paragraph

In [ ]:
const string FOLDER_PATH = "data/";
string[] files = Directory.GetFiles(FOLDER_PATH);
List<Chunk> chunks = [];

foreach (string file in files)
{
    if (file.Contains("water"))
    {
        Console.WriteLine(file);        
        var contents = File.ReadAllText(file).Split("\n\n");
        var fileName = Path.GetFileName(file);
        var id = 1;
        foreach(var content in contents)
        {
            // Add the chunk to the list
            var c = new Chunk($"{fileName}-{contents.Length}-{id}",content, fileName);
            Console.WriteLine(c);
            chunks.Add(c);
            id++;
        }
    }
}

### Save every chunk as a memory

In [ ]:
// Create an embedding generator to use for semantic memory.
foreach(var chunk in chunks)
{    
    await textMemory.SaveInformationAsync(MemoryCollectionName, id: chunk.Id, text: chunk.Text, description: chunk.File);
}

## Grounding

### Retrieve the memory based on a query

In [ ]:
var question = "What is the chemical composition of water?";

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
IAsyncEnumerable<MemoryQueryResult> queryResults =
                textMemory.SearchAsync(MemoryCollectionName, question, limit: 3, minRelevanceScore: 0.77);


### Find memories based on query, and collect the text in the memories to augment the prompt

In [ ]:
// Keep the text for the recalled memories
StringBuilder context = new StringBuilder();

#pragma warning disable CS8618,IDE0009,CA1051,CA1050,CA1707,CA2007,VSTHRD111,CS1591,RCS1110,CA5394,SKEXP0001,SKEXP0002,SKEXP0003,SKEXP0004,SKEXP0010,SKEXP0011,SKEXP0012,SKEXP0020,SKEXP0021,SKEXP0022,SKEXP0023,SKEXP0024,SKEXP0025,SKEXP0026,SKEXP0027,SKEXP0028,SKEXP0029,SKEXP0030,SKEXP0031,SKEXP0032,SKEXP0040,SKEXP0041,SKEXP0042,SKEXP0050,SKEXP0051,SKEXP0052,SKEXP0053,SKEXP0054,SKEXP0055,SKEXP0060,SKEXP0061,SKEXP0101,SKEXP0102
await foreach (MemoryQueryResult r in queryResults)
{
    // Append the text
    context.Append("Text:\n"+r.Metadata.Text+"\n\n"+"Source:\n"+r.Metadata.Description+"\n\n");
}

// Final augmented text
var promptContext = context.ToString();
Console.WriteLine($"User:\n{question}\n\nNearest results:\n{promptContext}")

## Process Prompt & Completion

### Create a SK function

In [ ]:
const string promptTemplate = "{{$input}}\n\nContext: ===\n{{$context}}\n===\n\nadd a source reference to the end of each sentence. e.g. Apple is a fruit [reference1.pdf][reference2.pdf]. Use only the provided text.";
var excuseFunction = kernel.CreateFunctionFromPrompt(promptTemplate, new OpenAIPromptExecutionSettings() { MaxTokens = 500, Temperature = 0.4, TopP = 1 });

### Submit the prompt and print the results

In [ ]:
string SplingLines(string longLine, int max_size =120)
{
    int currentLineLength = 0;
    StringBuilder sb = new StringBuilder();
    foreach (string word in longLine.Split(' '))
    {
        if (currentLineLength + word.Length >= max_size)
        {
            sb.AppendLine();
            currentLineLength = 0;
        }

        sb.Append(word + " ");
        currentLineLength += word.Length + 1;
    }
    return sb.ToString();
}

In [ ]:
var arguments = new KernelArguments()
        {
            ["input"] = question,
            ["context"] = promptContext
        };
var result = await kernel.InvokeAsync(excuseFunction, arguments);

Console.WriteLine($"Memories store: Redis Cache\n");
Console.WriteLine($"user:\n{question}\n");
Console.WriteLine($"assistant:\n{SplingLines(result.ToString())}");